In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data

if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    for dataset in range(1,7):
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../Data/Training{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            datas.append(data)
            header_datas.append(header_data)

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        print('Done.')

Dataset  1



Done.
Dataset  2



Done.
Dataset  3



Done.
Dataset  4



Done.
Dataset  5



Done.
Dataset  6



Done.


In [3]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

for i, Header_data in tqdm(enumerate(Header_datas)):
    codes = get_classes_from_header(Header_data)
    abbrs = ' '.join([get_abbr(int(code), Dx_map, Dx_map_unscored) for code in codes])
    for code, abbr in zip(codes, abbrs):
        if code in first_idx and first_idx[code] is None:
            first_idx[code] = i
        if code in first_idx_unscored and first_idx_unscored[code] is None:
            first_idx_unscored[code] = i
    encore = False
    for code in first_idx.keys():
        if first_idx[code] is None:
            encore = True
            break
#     for code in first_idx_unscored.keys():
#         if first_idx_unscored[code] is None:
#             encore = True
#             break
    if not encore:
        break

In [16]:
from signal_processing import myfilter, main_QRST
Codes = []
Q_locs = []
for idx in tqdm(range(0, 1000)): 
    if np.sum(Datas[idx]) != 0: # not all zeros
        codes = get_classes_from_header(Header_datas[idx])
        names = ', '.join([get_name(int(code), Dx_map, Dx_map_unscored) for code in codes])

        filtered_Data = myfilter(Datas[idx][:,1000:4000], 500, vis=False)

        # get the lead to apply Pan Tomkins
        Q_loc = main_QRST(filtered_Data, idx, '', '', names, fig2=False)

        # store
        Codes.append(codes)
        Q_locs.append(Q_loc)

In [17]:
len(Codes)

999

In [18]:
from global_vars import labels
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [20]:
data_labels = np.array([get_scored_class(codes, labels) for codes in Codes])
    

In [21]:
data_labels.shape

(999, 27)